In [141]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [143]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 16

embeddings = CustomEmbedding(vocab_size = vocab_size + 3, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-0.5781,  0.9327,  0.5729,  0.6851,  0.2089, -1.3305, -0.0123, -0.8313,
          1.9145,  2.0170,  1.6838,  0.5228, -1.3631, -0.1331, -0.5326, -0.4767],
        [-1.3818,  2.3472,  0.8374, -0.0409, -0.5615,  0.5074, -0.4298, -0.0503,
          1.6774,  0.9718, -0.2240, -0.0522,  0.4305, -0.8755, -1.4340,  1.2245]],
       grad_fn=<EmbeddingBackward0>)


In [ ]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 32
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, 10000, [i for i in range(10)])
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [ ]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 5,  7,  5,  9,  0,  2, 10, 10, 10, 10],
        [ 1,  0,  8,  0,  5,  9,  0,  1,  8,  4],
        [ 0,  4,  8,  5,  2,  3, 10, 10, 10, 10],
        [ 8,  3,  4,  4,  4,  0,  8, 10, 10, 10],
        [ 7,  9,  2,  1,  6,  0, 10, 10, 10, 10],
        [ 6,  0,  6,  6,  6,  2,  7,  0,  4, 10],
        [ 0,  1, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 4,  7, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 1,  3,  8, 10, 10, 10, 10, 10, 10, 10],
        [ 1,  7,  3,  6,  7,  4,  1, 10, 10, 10],
        [ 3,  8,  2,  1, 10, 10, 10, 10, 10, 10],
        [ 3,  7,  9,  1,  6,  0,  5,  5,  9, 10],
        [ 5,  2,  8,  9,  8,  0,  5, 10, 10, 10],
        [ 5,  8, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 2,  5,  8,  6, 10, 10, 10, 10, 10, 10],
        [ 9,  2,  2,  7,  5,  7,  4,  5,  1, 10],
        [ 2,  7,  6,  8,  4,  7, 10, 10, 10, 10],
        [ 2,  7,  1,  5,  6,  2,  4,  0, 10, 10],
        [ 3,  0,  5,  0,  6,  4,  2, 10, 10, 10],
        [ 0,  8,  8, 10, 10, 10, 10, 10, 10, 10],


In [146]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [147]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """

    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for i, ((source, target), label) in tqdm(enumerate(dataloader)):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, vocab_size), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences.item() / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens.item() / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    
    model.eval()

In [ ]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

validation_losses = list()
validation_sequence_accuracies = list()
validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    # validation_loss = run_validation_loop(dataloader, model, loss_fn)
    # validation_losses.append(validation_loss)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(validation_losses)
print(validation_sequence_accuracies)
print(validation_token_accuracies)

313it [00:02, 132.39it/s]
313it [00:02, 139.02it/s]
313it [00:02, 141.37it/s]
313it [00:02, 139.09it/s]
313it [00:02, 139.64it/s]
313it [00:02, 142.65it/s]
313it [00:02, 140.85it/s]
313it [00:02, 142.84it/s]
313it [00:02, 140.02it/s]
313it [00:02, 141.48it/s]

[4.637418487930298, 2.723798373031616, 2.3811994663238525, 2.2533450035095215, 2.066298963546753, 1.928682300376892, 1.7139234970092774, 1.983107015800476, 1.8100524277687073, 1.660180486011505]
[0.0027, 0.0088, 0.0121, 0.0165, 0.0192, 0.0229, 0.0284, 0.0229, 0.0277, 0.0282]
[0.47013838310269485, 0.5406154294881144, 0.5519300801165331, 0.5564037705634007, 0.5648964863682753, 0.5685489739484791, 0.575278505897772, 0.5678714493809177, 0.5738677734090578, 0.5777036875091095]
